In [24]:
import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import BoundaryNorm
import matplotlib.patches as mpatches
import seaborn as sns
import pandas as pd
import iris
import iris.quickplot as qplt
import iris.plot as iplt
import cartopy.crs as ccrs
import datetime
import warnings

# Set the global warning filter to ignore all warnings
warnings.simplefilter("ignore")
%matplotlib tk

In [25]:
def make_combined_df(date, members):
    date_label = date.strftime("%Y%m%d")
    bsiso_archive_dir = os.path.join('/project/MJO_GCSS/SEA_monitoring/processed_BSISO_data/mogreps/bsiso_new/', 
                                     date_label)
    dfs = []
    for m, mem in enumerate(members):
        bsiso_index_file_name = os.path.join(bsiso_archive_dir,
                                             f'BSISO.{date_label}.fcast.{mem}.txt')
        # Concatenate all DataFrames in the list along rows
        pcs = pd.read_csv(bsiso_index_file_name)
        # Append the computed PCs for the current member to the list
        dfs.append(pcs)

    combined_df = pd.concat(dfs, ignore_index=True)
    return combined_df

In [26]:
def make_ens_mean_df(combined_df):
    ens_mean_df = combined_df.groupby('date', as_index=False)[['PC1', '8_phases', 'PC2', 'amp']].mean()
    return ens_mean_df

In [27]:
def read_phase_cubes():
    phase_cubes = []
    for phase in range(1, 9):
        file_name = os.path.join('/project/MJO_GCSS/SoutheastAsia_data/GPM/data_for_composites', f'pr_data_phase{phase}_anomaly.nc')
        phase_cubes.append(iris.load_cube(file_name))
    return phase_cubes

In [184]:
date = datetime.datetime(2024, 5, 4)
nanalysis2write = 40
members = [f"{mem:03}" for mem in range(1,36)]
combined_df = make_combined_df(date, members)
ens_mean_df  = make_ens_mean_df(combined_df)
combined_df

,Year,Month,Day,date,PC1,PC2,8_phases,amp,4_phases,mem,label
0,2024,3,26,2024-03-26,0.830703,-0.501721,4,0.970459,3,1,analysis
1,2024,3,27,2024-03-27,-0.679565,-1.816698,2,1.939639,2,1,analysis
2,2024,3,28,2024-03-28,-1.030148,-1.301836,2,1.660115,2,1,analysis
3,2024,3,29,2024-03-29,0.029823,-0.261464,3,0.263159,2,1,analysis
4,2024,3,30,2024-03-30,0.415502,-0.133673,4,0.436475,3,1,analysis
...,...,...,...,...,...,...,...,...,...,...,...
1640,2024,5,7,2024-05-07,-1.104267,2.449174,7,2.686607,4,35,forecast
1641,2024,5,8,2024-05-08,-0.161280,2.487130,7,2.492354,4,35,forecast
1642,2024,5,9,2024-05-09,0.467911,1.936510,6,1.992238,4,35,forecast
1643,2024,5,10,2024-05-10,0.109718,1.320497,6,1.325047,4,35,forecast


In [185]:
phase_cubes = read_phase_cubes()

In [201]:
sns.set_theme()
# Desired pixel dimensions
pixel_width = 500
pixel_height = 500

# DPI (dots per inch)
dpi = 75  # This is a common default but can be adjusted to your needs

# Convert pixel dimensions to inches for matplotlib
inch_width = pixel_width / dpi
inch_height = pixel_height / dpi

# Create the figure with the computed size in inches
fig = plt.figure(figsize=(inch_width, inch_height), dpi=dpi)
#fig = plt.figure(figsize=(10, 10))
x = 4
y = 0.707107
linewidth = 0.25

# Original figure
ax = fig.add_subplot(111, aspect='equal')
ax.plot([-x, -y], [-x, -y], 'k', lw=linewidth)
ax.plot([y, x], [y, x], 'k', lw=linewidth)
ax.plot([-x, -y], [x, y], 'k', lw=linewidth)
ax.plot([y, x], [-y, -x], 'k', lw=linewidth)
ax.plot([-x, -1], [0, 0], 'k', lw=linewidth)
ax.plot([1, x], [0, 0], 'k', lw=linewidth)
ax.plot([0, 0], [-x, -1], 'k', lw=linewidth)
ax.plot([0, 0], [1, x], 'k', lw=linewidth)
ax.set_ylim(-x, x)
ax.set_xlim(-x, x)
circle = mpatches.Circle((0, 0), 1, fc="white", ec="k", lw=linewidth)
ax.add_patch(circle)

# Enable grid
ax.grid(True)

# Remove tick labels but keep the grid
ax.set_xticklabels([])
ax.set_yticklabels([])
# Optionally, remove the axis entirely if you want a clean image with just the plot
#ax.axis('off')  # This command turns off the axes completely, including the borders

# Set aspect of the plot to be equal, to keep the image from distorting
ax.set_aspect('equal')

width = 0.25
height = 0.15
positions_dict = {
    1: [0.05, 0.3, width, height],
    2: [0.25, 0.1, width, height],
    3: [0.525, 0.1, width, height],
    4: [0.725, 0.3, width, height],
    5: [0.725, 0.55, width, height],
    6: [0.525, 0.775, width, height],
    7: [0.25, 0.775, width, height],
    8: [0.05, 0.55, width, height]
}

# Draw subpanels on top of the original figure
for i, phase in enumerate(range(1,9)):
    subax = fig.add_axes(positions_dict[phase], projection=ccrs.PlateCarree())
    xp = iplt.pcolormesh(phase_cubes[i], axes=subax, norm=norm, cmap=cmap, alpha=0.75)
    subax.coastlines()
    subax.set_title(f'{phase}', fontsize=14, alpha=0.7)

plt.tight_layout()
plt.savefig('../data/BSISO_Phases_BG.png')
plt.close()
#plt.show()

In [187]:
from PIL import Image
import numpy as np

# Load the image
img = Image.open('BSISO_Phases_BG.png')

In [190]:
# Set up the plot
fig, ax = plt.subplots(figsize=(10, 10))
xx = 4.15
ax.imshow(img, extent=[-xx, xx, -xx, xx])  # Adjust these values based on your actual data
#ax.plot(range(2),range(2))
x = 4
ax.set_ylim(-x, x)
ax.set_xlim(-x, x)
# Remove grid
ax.grid(True)  # This disables the grid

# Remove axis labels and tick marks
#ax.set_xticks([])  # No x ticks
#ax.set_yticks([])  # No y ticks

# Optionally, remove the axis entirely if you want a clean image with just the plot
ax.axis('on')  # This command turns off the axes completely, including the borders

# Set aspect of the plot to be equal, to keep the image from distorting
ax.set_aspect('equal')

cmap = plt.get_cmap('BrBG')
levels=np.linspace(-3, 3, 11)
norm = BoundaryNorm(levels, ncolors=cmap.N, clip=True)
for mem in members:
    df = combined_df.loc[combined_df.mem==int(mem)].iloc[-8:]
    ax.scatter(df.PC1, df.PC2, s=df.amp*50, color='grey', alpha=0.2)
    ax.plot(df.PC1, df.PC2, color='grey', alpha=0.2)

analysis = ens_mean_df.iloc[-15:-7]
ax.scatter(analysis.PC1, analysis.PC2, s=analysis.amp*20, alpha=0.8)
ax.plot(analysis.PC1, analysis.PC2, color='grey', linewidth=3, alpha=0.8)

forecast = ens_mean_df.iloc[-8:]
ax.scatter(forecast.PC1, forecast.PC2, s=forecast.amp*50, color='blue', alpha=0.8)
ax.plot(forecast.PC1, forecast.PC2, color='blue', linewidth=3, alpha=0.8)
for i, row in forecast.iterrows():
    ax.text(row.PC1+0.075, row.PC2+0.075, str(row.date), ha="left", va="center", 
            bbox=dict(boxstyle="round", alpha=0.4, ec=(1., 0.5, 0.5), fc=(1., 0.8, 0.8),))
plt.title(f"BSISO Index forecast: {date.strftime('%Y-%m-%d')}")
plt.tight_layout()
plt.show()

In [172]:
date

datetime.datetime(2024, 5, 4, 0, 0)

In [127]:
ens_mean_df.date

32    2024-04-27
33    2024-04-28
34    2024-04-29
35    2024-04-30
36    2024-05-01
37    2024-05-02
38    2024-05-03
39    2024-05-04
40    2024-05-05
41    2024-05-06
42    2024-05-07
43    2024-05-08
44    2024-05-09
45    2024-05-10
46    2024-05-11
Name: date, dtype: object